In [1]:
%load_ext autoreload
%autoreload 2

# Automatic differentiation of RNA circuit dynamics

# Imports

In [2]:
import numpy as np
import jax
import jax.numpy as jnp
from jax import random
import optax
import equinox
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import diffrax as dfx
from typing import List

from functools import partial
import os
import sys

from bioreaction.simulation.simfuncs.basic_de import one_step_de_sim_expanded

jax.config.update('jax_platform_name', 'gpu')

if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)


os.environ['EQX_ON_ERROR']='breakpoint'
os.environ['JAX_DISABLE_JIT']='1'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

np.random.seed(0)
jax.devices()

[cuda(id=0), cuda(id=1)]

In [3]:
from synbio_morpher.srv.parameter_prediction.simulator import make_piecewise_stepcontrol
from synbio_morpher.utils.misc.type_handling import flatten_listlike
from synbio_morpher.utils.modelling.physical import eqconstant_to_rates, equilibrium_constant_reparameterisation
from synbio_morpher.utils.modelling.deterministic import bioreaction_sim_dfx_expanded
from synbio_morpher.utils.modelling.solvers import get_diffrax_solver, make_stepsize_controller, simulate_steady_states
from synbio_morpher.utils.results.analytics.timeseries import calculate_adaptation, compute_peaks, compute_adaptability_full
from bioreaction.simulation.simfuncs.basic_de import bioreaction_sim, bioreaction_sim_expanded, one_step_de_sim_expanded


# Set up test environment

In [4]:
def make_species_bound(species_unbound):
    return sorted(set(flatten_listlike([['-'.join(sorted([x, y])) for x in species_unbound] for y in species_unbound])))


# RNA circuit settings
species_unbound = ['RNA_0', 'RNA_1', 'RNA_2']
species_bound = make_species_bound(species_unbound)
species = species_unbound + species_bound
species_signal = ['RNA_0']
species_output = ['RNA_2']
# species_output = ['RNA_1', 'RNA_2']
species_nonsignal = [s for s in species_unbound if s not in species_signal]
idxs_signal = np.array([species.index(s) for s in species_signal])
idxs_output = np.array([species.index(s) for s in species_output])
idxs_unbound = np.array([species.index(s) for s in species_unbound])
idxs_bound = np.array([species.index(s) for s in species_bound])
signal_onehot = jnp.array([1 if s in idxs_signal else 0 for s in np.arange(len(species))])
signal_onehot_inv = (signal_onehot == 0) * 1.0

# Circuit parameters
n_circuits = 200
n_circuits_display = 30
key = random.PRNGKey(0)
N0 = 200
y00 = np.array([[N0] * len(species_unbound) + [0] * len(species_bound)]).astype(np.float32)
y00 = np.repeat(y00, repeats=n_circuits, axis=0)

# Dynamic Simulation parameters
k_a = 0.00150958097
signal_target = 2
t0 = 0
t1 = 0.001
ts = np.linspace(t0, t1, 500)
tmax = 1000
dt0 = 0.005555558569638981
dt1_factor = 5
dt1 = dt0 * dt1_factor
max_steps = 16**4 * 10
use_sensitivity_func1 = False
sim_method = 'Dopri8'
stepsize_controller = 'adaptive'
threshold_steady_state = 0.01
batch_size = 10
n_steps = 10                   # Number of optimization steps

# Reactions
energies = random.normal(key, (n_circuits, len(np.tril_indices(len(species_unbound))[0])))
energies = np.interp(energies, (energies.min(), energies.max()), (-25, 0))
eqconstants = jax.vmap(partial(equilibrium_constant_reparameterisation, initial=N0))(energies)
forward_rates, reverse_rates = eqconstant_to_rates(eqconstants, k_a)
forward_rates = forward_rates[0]

inputs = np.array([
    [2, 0, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 2, 0, 0, 0, 0, 0, 0],
], dtype=np.float64)
outputs = np.array([
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
], dtype=np.float64)

# ODE functions

In [5]:
sim_func = partial(bioreaction_sim_dfx_expanded,
                   t0=t0, t1=t1, dt0=dt0,
                   forward_rates=forward_rates,
                   inputs=inputs,
                   outputs=outputs,
                   solver=get_diffrax_solver(sim_method),
                   stepsize_controller=make_stepsize_controller(t0, t1, dt0, dt1, choice=stepsize_controller))

In [6]:
def obj_s(deriv, y, y0):
    # return - jnp.divide(jnp.power(y - y0, 2), y0)
    return deriv * jnp.power(jnp.divide(y - y0, y0), 2)


def obj_p(y, y0):
    # return jnp.divide(jnp.power(y - y0, 2), y0)
    return jnp.power(jnp.divide(y - y0, y0), 2)


def objective_sp(s, p, ttot, idxs_output):
    s_objective = (s ** 2)[..., idxs_output].sum() / ttot
    p_objective = - (p ** 2)[..., idxs_output].sum() / ttot
    return s_objective + p_objective


def simulate_signal(y0, reverse_rates):
    sol = partial(sim_func, return_as_sol=True, 
                  saveat=dfx.SaveAt(ts=jnp.linspace(t0, t1, 500), dense=True))(
        y0=y0, reverse_rates=reverse_rates)
    ddy_dt = jax.vmap(sol.derivative)(sol.ts)
    y_sens = obj_s(ddy_dt, sol.ys, y0)
    y_prec = obj_p(sol.ys, y0)
    return sol.ts, (sol.ys, ddy_dt, y_sens, y_prec)


def simulate(y0, params):
    """ (y11, ddys1, y_sens, y_prec) = ys1 """
    ts0, ys0 = jax.vmap(partial(sim_func, saveat=dfx.SaveAt(ts=jnp.linspace(t0, t1, 500))))(y0=y0, reverse_rates=params)
    y01 = (jnp.array(ys0[:, -1]) * signal_onehot * signal_target) + (jnp.array(ys0[:, -1]) * signal_onehot_inv)
    # y01[:, idxs_signal] = signal_target * y01[:, idxs_signal]
    ts1, ys1 = jax.vmap(simulate_signal)(y0=y01, reverse_rates=params)
    # (y11, ddys1, y_sens, y_prec) = ys1
    return (ts0, ys0), (ts1, ys1)

In [ ]:
y0 = y00[:10]
params = reverse_rates[:10]

(ts0, ys0), (ts1, ys1) = simulate(y0, params=params)

2024-11-01 17:11:29.942368: W external/xla/xla/service/hlo_rematerialization.cc:3005] Can't reduce memory use below 11.47GiB (12315157267 bytes) by rematerialization; only reduced to 12.46GiB (13380271664 bytes), down from 12.46GiB (13380271664 bytes) originally
2024-11-01 17:11:40.289058: W external/tsl/tsl/framework/bfc_allocator.cc:482] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.92GiB (rounded to 5284823040)requested by op 
2024-11-01 17:11:40.289137: W external/tsl/tsl/framework/bfc_allocator.cc:494] __________________________________________________________________________________________________*_
E1101 17:11:40.289166  136541 pjrt_stream_executor_client.cc:2985] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5284823040 bytes.


XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 5284823040 bytes.

## Visualise ODE sim

In [ ]:
visualise = False

if visualise:
    y0 = y00[:10]
    params = reverse_rates[:10]

    (ts0, ys0), (ts1, ys1) = simulate(y0, params)
    (y11, ddys1, y_sens, y_prec) = ys1

    n_plots=6
    plt.figure(figsize=(5*n_plots, 5))
    titles = ['y', 'ddy', 'y_sens', 'y_prec']
    i_circ = 5
    for i, title in enumerate(titles):
        ax = plt.subplot(1, n_plots, i+1)
        plt.plot(ts, ys1[i][i_circ, :, idxs_output].T)
        plt.title(title)
        plt.legend(species_output)
    ax = plt.subplot(1, n_plots, 5)
    plt.plot(ts, (ys1[2][i_circ, :, idxs_output].T * ys1[3][i_circ, :, idxs_output].T))
    plt.title('y_sens * y_prec')
    plt.legend(species_output)

    ax = plt.subplot(1, n_plots, 6)
    plt.plot(ts, ((ys1[1] ** 2)[i_circ, :, idxs_output].T * ys1[3][i_circ, :, idxs_output].T))
    plt.legend(species_output)
    plt.title('ddy^2 * y_prec')

    print(y_sens.sum()/ t1, y_prec.sum()/t1)

In [ ]:
if visualise:
    ts = jnp.concatenate([ts0, ts1 + ts0.max()], axis=1)[0]
    ys = jnp.concatenate([ys0, ys1[0]], axis=1)
    loss = jax.vmap(partial(objective_sp,
                            ttot=t1-t0, idxs_output=idxs_output))(y_sens, y_prec)

    nr = int(np.ceil(np.sqrt(len(params))))
    nc = int(np.ceil(np.sqrt(len(params))))

    plt.figure(figsize=(5*nc, 5*nr))
    for i in range(len(params)):
        plt.subplot(nr, nc, i+1)
        plt.plot(ts1[0], y11[i][..., idxs_output])
        
        plt.title(f'Circuit {i}: loss = {loss[i]}')

In [ ]:
# # Define the ODE function
# def ode_func(t, y, params):
#     return params.sum(axis=-1) - y  # (y * params).sum(axis=-1) - y


# def smooth_max(a, b, epsilon=1e-2):
#     """ Smooth approximation for maximum """
#     return epsilon * jnp.log(jnp.exp(a / epsilon) + jnp.exp(b / epsilon))


# def smooth_max2(a, b):
#     """ Smooth maximum approximation using softplus"""
#     return jnp.log1p(jnp.exp(a - b)) + b  # Softplus-based approximation


# def max_value_ode(t, y, args):
#     """ `args` is a tuple (params, max_val) """
#     params, epsilon = args
#     max_val = y[1]
#     dy_dt = ode_func(t, y[0], params)
#     # max_val = jnp.max(jnp.vstack([max_val, y[0]]), axis=0)
#     max_val = smooth_max(max_val, y[0], epsilon)
#     # return dy_dt, jnp.sin(y[0])
#     return dy_dt, max_val


# def ode_sp(t, y, args):
#     params, y0 = args
#     # y_t, y_sens, y_prec = y
#     y_t = y
#     dy_dt = sim_func(t, y_t, args, reverse_rates=params)
#     y_sens = obj_s(dy_dt, y_t, y0)
#     y_prec = obj_p(y_t, y0)
#     return dy_dt #, y_sens, y_prec


# # Modified function to solve the ODE and track the maximum value with params
# def solve_ode_and_track_max(y0, params, t0, t1, dt0, dt1, stepsize_controller, max_steps=1000, idx_output=-1):

#     # Solve the ODE with an auxiliary variable to track the max
#     solution = dfx.diffeqsolve(
#         dfx.ODETerm(ode_sp),
#         solver=get_diffrax_solver(sim_method),
#         t0=t0,
#         t1=t1,
#         dt0=dt0,
#         y0=y0,
#         # y0=(y0, y0, y0),
#         args=(params, y0),  # Pass params and initial max as args
#         saveat=dfx.SaveAt(ts=jnp.linspace(t0, t1, max_steps)),
#         stepsize_controller=make_stepsize_controller(t0, t1, dt0, dt1,
#                                                      choice=stepsize_controller)
#         # adjoint=dfx.BacksolveAdjoint()  # This enables differentiation
#     )

#     y_sens, y_prec = solution.ys[1][-2:]  # Get the max value reached
#     return solution, y_sens[idx_output], y_prec[idx_output]


# def solve_naive(y0, params, t0, t1, dt0, max_steps=10000, idx_output=-1):

#     y, y_sens, y_prec = y0, y0, y0
#     # , np.zeros((max_steps, len(y0))), np.zeros((max_steps, len(y0)))
#     ys, ys_sens, ys_prec = np.zeros((3, max_steps, len(y0)))
#     for i_t, tstep in enumerate(np.linspace(t0, t1, max_steps)):
#         dy_dt, y_sens, y_prec = ode_sp(
#             tstep, (y, y_sens, y_prec), (params, y0))
#         y = y + dy_dt * dt0
#         ys[i_t] = y
#         ys_sens[i_t] = y_sens
#         ys_prec[i_t] = y_prec
#     return ys, ys_sens, ys_prec

# Optimisation functions

In [ ]:
# @jax.jit
def loss_fn(params, y0, t0, t1):
    """ Define a loss function to optimize params """
    (ts0, ys0), (ts1, ys1) = simulate(y0, params)
    (y11, ddys1, y_sens, y_prec) = ys1

    loss = jax.vmap(
        partial(objective_sp, ttot=t1-t0, idxs_output=idxs_output)
        )(y_sens, y_prec)
    
    return loss

# @jax.jit
def batch_loop(params_b, y0b, t0, t1, optimiser, opt_state):

    loss_b, grads_b = jax.value_and_grad(
        partial(loss_fn, t0=t0, t1=t1))(params_b, y0b)
    updates, opt_state = optimiser.update(grads_b, opt_state)
    params_b = optax.apply_updates(params_b, updates)

    return loss_b, grads_b, params_b


# @jax.jit
@equinox.filter_jit
def optimise_step(params, opt_state, y0, t0, t1, optimiser, batch_size):
    def extend_batch(xs, xsb):
        if xs is None:
            xs = xsb
        else:
            xs = jnp.concatenate([xs, xsb])
        return xs

    loss, grads = (None, None)
    for batch_step in range(0, len(params), batch_size):
        print(f"Batch step: {batch_step}/{len(params)}")
        i0, i1 = batch_step, batch_step + batch_size
        params_b, y0b = (params[i0:i1], y0[i0:i1])

        loss_b, grads_b, params_b = batch_loop(
            params_b, y0b, t0, t1, optimiser, opt_state)

        params = jax.ops.index_update(params, jax.ops.index[i0:i1], params_b)
        loss = extend_batch(loss, loss_b)
        grads = extend_batch(grads, grads_b)
    return params, opt_state, loss, grads

In [ ]:
params = reverse_rates

# Set up optax optimiser
optimiser = optax.adam(learning_rate=0.01)
opt_state = optimiser.init(params)

# Optimization loop
loss_all = np.zeros(n_steps)
grads_all = np.zeros((n_steps, len(params)))
for i in range(n_steps):
    print(f'Step {i}')
    params, opt_state, current_loss, current_grads = optimise_step(
        params, opt_state, y00, t0, t1, optimiser, batch_size)
    if i % 100 == 0:
        print(f"Step {i}, Loss: {current_loss}, Params: {params}")

print(f"Optimized Params: {params}")

# solution_steps, y_sens, y_prec = solve_ode_and_track_max(
#     ode_func, y00, t0, t1, params)

Step 0
Batch step: 0/200


NotImplementedError: Cannot use `adjoint=BacksolveAdjoint()` with `saveat=SaveAt(steps=True)` or saveat=SaveAt(dense=True)`.

In [ ]:
# Comparison of Max's
x1 = jnp.ones(10) * 2.
x2 = jnp.linspace(0, 2, 10)
max_val = smooth_max(x1, x2, epsilon=1e-1)
# max_val = smooth_max2(x1, x2)
max_val, jnp.maximum(x1, x2)